In [1]:
import sys
from pyspark import SparkConf, SparkContext, SQLContext
import pyspark.sql.functions as func
from pyspark.sql.session import SparkSession
#from pyspark.mllib.linalg.distributed import CoordinateMatrix, MatrixEntry
from itertools import islice

In [2]:
conf = SparkConf()

In [3]:
#sc = SparkContext(conf=conf)
sc = SparkContext(conf=conf).getOrCreate()

In [4]:
sqlCon = SQLContext(sc)

In [5]:
spark = SparkSession(sc)

In [6]:
segments = spark.read.option("sep", "\t").option("header","true").csv("amazon_reviews_us_Books.tsv")

In [ ]:
segments.show(5)

In [39]:
segments.limit(5).toPandas()

,marketplace,customer_id,review_id,product_id,product_parent,product_title,product_category,star_rating,helpful_votes,total_votes,vine,verified_purchase,review_headline,review_body,review_date
0,US,12076615,RQ58W7SMO911M,0385730586,122662979,Sisterhood of the Traveling Pants (Book 1),Books,4,2,3,N,N,this book was a great learning novel!,this boook was a great one that you could lear...,2005-10-14
1,US,12703090,RF6IUKMGL8SF,0811828964,56191234,The Bad Girl's Guide to Getting What You Want,Books,3,5,5,N,N,Fun Fluff,If you are looking for something to stimulate ...,2005-10-14
2,US,12257412,R1DOSHH6AI622S,1844161560,253182049,"Eisenhorn (A Warhammer 40,000 Omnibus)",Books,4,1,22,N,N,this isn't a review,never read it-a young relative idicated he lik...,2005-10-14
3,US,50732546,RATOTLA3OF70O,0373836635,348672532,Colby Conspiracy (Colby Agency),Books,5,2,2,N,N,fine author on her A-game,Though she is honored to be Chicago Woman of t...,2005-10-14
4,US,51964897,R1TNWRKIVHVYOV,0262181533,598678717,The Psychology of Proof: Deductive Reasoning i...,Books,4,0,2,N,N,Execellent cursor examination,Review based on a cursory examination by Unive...,2005-10-14


In [7]:
tmp = segments.select("customer_id", "product_id", "star_rating") 

In [8]:
distinct_users = tmp.select("customer_id").distinct().count()
#min_prod = tmp.

In [9]:
print(distinct_users)
min_user_thres = 100

1502380


In [10]:
rating_count = tmp.groupBy("product_id").agg(func.countDistinct("customer_id"))
rating_count.show()

+----------+---------------------------+
|product_id|count(DISTINCT customer_id)|
+----------+---------------------------+
|0688075398|                          3|
|0791455041|                          1|
|0970093349|                          9|
|0765702428|                          2|
|3000439978|                          1|
|0140177558|                          1|
|0842329129|                       1291|
|0618434194|                          4|
|1880656183|                         18|
|1413724337|                          1|
|0395977894|                        367|
|0399113290|                          1|
|1401300871|                         38|
|0782123406|                          9|
|0596006748|                         60|
|0553800914|                         48|
|0679451250|                          7|
|0142003425|                          5|
|0786865059|                        162|
|0394720954|                         13|
+----------+---------------------------+
only showing top

In [11]:
rating_count= rating_count.withColumnRenamed("count(DISTINCT customer_id)", "count_distinct_cust")
rating_count.show()

+----------+-------------------+
|product_id|count_distinct_cust|
+----------+-------------------+
|0688075398|                  3|
|0791455041|                  1|
|0970093349|                  9|
|0765702428|                  2|
|3000439978|                  1|
|0140177558|                  1|
|0842329129|               1291|
|0618434194|                  4|
|1880656183|                 18|
|1413724337|                  1|
|0395977894|                367|
|0399113290|                  1|
|1401300871|                 38|
|0782123406|                  9|
|0596006748|                 60|
|0553800914|                 48|
|0679451250|                  7|
|0142003425|                  5|
|0786865059|                162|
|0394720954|                 13|
+----------+-------------------+
only showing top 20 rows



In [12]:
prod_list = rating_count.filter(rating_count["count_distinct_cust"]>min_user_thres) \
                        .select("product_id").distinct()
prod_list.show()

+----------+
|product_id|
+----------+
|0842329129|
|0395977894|
|0786865059|
|081257558X|
|0812550285|
|0767905385|
|1575666960|
|0060842423|
|1400050308|
|0843952016|
|034540288X|
|0064407314|
|0670033944|
|0316168815|
|034544356X|
|0895261405|
|044021145X|
|1576734587|
|0684853515|
|0962067164|
+----------+
only showing top 20 rows



In [13]:
prod_list.count()

1622

In [14]:
tmp.select("product_id").distinct().count()

779733

In [ ]:
#synctax check
prod_list = list(prod_list.select("product_id").toPandas()["product_id"])

In [15]:
tmp = tmp.withColumn("star_rating", tmp["star_rating"].cast("double"))
tmp.show(5)

+-----------+----------+-----------+
|customer_id|product_id|star_rating|
+-----------+----------+-----------+
|   12076615|0385730586|        4.0|
|   12703090|0811828964|        3.0|
|   12257412|1844161560|        4.0|
|   50732546|0373836635|        5.0|
|   51964897|0262181533|        4.0|
+-----------+----------+-----------+
only showing top 5 rows



In [28]:
cust_prod_rating = tmp.filter(tmp['product_id'].isin(prod_list['product_id']))
#                               .groupBy("customer_id").pivot("product_id").avg("star_rating"))
cust_prod_rating.show(5)

+-----------+----------+-----------+
|customer_id|product_id|star_rating|
+-----------+----------+-----------+
|   12076615|0385730586|        4.0|
|   12703090|0811828964|        3.0|
|   12257412|1844161560|        4.0|
|   50732546|0373836635|        5.0|
|   51964897|0262181533|        4.0|
+-----------+----------+-----------+
only showing top 5 rows



In [52]:
cust_prod_rating_filtered=cust_prod_rating.limit(2000)
# cust_prod_rating_filtered.show(1000)

In [53]:
cust_prod_rating_filtered_pivot=cust_prod_rating_filtered.groupBy("customer_id").pivot("product_id").avg("star_rating")

In [54]:
cust_prod_rating_filtered_pivot.toPandas()

,customer_id,0006483895,0027440702,0027893901,0028619641,0028643771,0029117062,0029146739,0029148510,0060194111,...,B0009A0GXW,B0009W8L6Y,B000A176OM,B000AXRTSS,B000B5RXKO,B000B5RXSG,B000B86FLY,B000B8NK3U,B000BAT5UU,B000BL30QO
0,12076615,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,12703090,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,12257412,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,50732546,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,51964897,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,31048862,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,53000124,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,29732693,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,48541186,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,15280864,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [55]:
# spark.conf.set("spark.sql.pivotMaxValues", 800000)
# ratings_pivot = cust_prod_rating.groupBy("customer_id").pivot("product_id").avg("star_rating")

In [ ]:
cust_prod_rating2= cust_prod_rating.collect()
cust_prod_rating2.show(5)